In [1]:
#import os
#os.environ['ESMFMKFILE'] = '/home/malasawedah/.conda/envs/test2/lib/esmf.mk'

# Now you can import the ESMF library
#import ESMF


In [2]:
#!conda install -c conda-forge xesmf -y #>= 0.7.1

In [3]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#import xvec
import shapely
from shapely import Point, box
import esmpy
import xagg as xa
import rioxarray

# supress warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

Found esmf.mk at: /home/cgoehner/.conda/envs/xesmf_env/lib/esmf.mk


In [4]:
xa.aggregate

<function xagg.core.aggregate(ds, wm, impl='for_loop', stat='mean', skipna=True, interpolate_NaN=False, silent=False)>

In [5]:
#!pip install dask

In [6]:
aoi = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/01_Ground_Data/Versuchsflächen_DRI2.shp')
# grid5 = pd.read_csv('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/polygons_grid_5')
#chunks = {'t': 1, 'x': 100, 'y': 100}
#s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc', chunks= chunks)
s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/dsc/grid4/S1_grid4_3/openEO_filtered.nc').sel(t = "2021-05-13")
# s1 = xr.open_dataset("/home/cgoehner/Documents/extract_aggr/raster.tif").sel(band = 1)
# s1 = xr.open_dataset("/home/cgoehner/Documents/extract_aggr/raster_VV.tif").sel(band = 1)

In [7]:
gdf_grid5 = aoi[5:6]
gdf_grid5

,Test_site,Farm,Plot,Area,geometry
5,Laurein,Kausen\n,L1,5160,"POLYGON ((11.07948 46.45398, 11.07985 46.45343..."


In [8]:
s1 = s1.rio.write_crs("EPSG:32632", inplace=True).rio.reproject('EPSG:4326')
# s1 = s1.rio.write_crs("EPSG:4326")

In [9]:
# statistics highly dependent on selected crs: pixel sizes vary with crs
test = s1
pixel = box(test.x[0].values, test.y[0].values, test.x[1].values, test.y[1].values)
pixel = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[pixel]).to_crs("6931") # 6931 for northern hemishpere
# pixel = pixel.to_crs("+proj=cea +units=m")
# pixel = pixel.to_crs({'proj':'cea'}) 

print("Pixel size:", pixel.geometry.area[0], "m².\nShould be 100 m² as in EPSG:32632.")

Pixel size: 122.24672158630038 m².
Should be 100 m² as in EPSG:32632.


In [10]:
# grid5['geometry'] = grid5['geometry'].apply(lambda x: shapely.wkt.loads(x))
# gdf_grid5 = gpd.GeoDataFrame(grid5, geometry='geometry')
# gdf_grid5.crs = 'EPSG:4326'
# gdf_grid5.head(3)

In [11]:
#chunks = {'t': 1, 'x': 100, 'y': 100}
#s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc', chunks= chunks)
#s1_wgs84 = s1_wgs84.chunk({'t': 1, 'x': 50, 'y': 50})

In [12]:
%%time
weightmap = xa.pixel_overlaps(s1, gdf_grid5)

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!
CPU times: user 183 ms, sys: 17.8 ms, total: 201 ms
Wall time: 207 ms


In [13]:
ds = s1
wm = weightmap
# impl='for_loop'
silent=False

In [14]:
# print("Max pixel size:", list(map(max, list(map(sum, wm.agg.poly_area))))[0],
#       "\nSum of rel_area should add up to 1:", list(map(sum, list(map(sum, wm.agg.rel_area))))[0],
#       "\nPixel count:", list(map(sum, list(map(sum, wm.agg.rel_area.values))))[0] / list(map(max, list(map(max, wm.agg.rel_area.values))))[0])

In [15]:
wm.agg

,Test_site,Farm,Plot,Area,poly_idx,rel_area,poly_area,pix_idxs,coords
0,Laurein,Kausen\n,L1,5160,0,"[[8.739657751400218e-05, 0.0049163659442638214...","[[0.4509793153360937, 25.369178182412945, 116....","[17, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 4...","[(46.45326555050074, 11.078368305718406), (46...."


In [16]:
test = xa.aggregate(ds, wm, stat = ["count", "sum", "mean", "median", "std", "min", "max"], interpolate_NaN = False, silent = True).to_dataset()

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful


In [17]:
test

<xarray.Dataset>
Dimensions:    (poly_idx: 1)
Coordinates:
  * poly_idx   (poly_idx) int64 0
Data variables: (12/19)
    Test_site  (poly_idx) object 'Laurein'
    Farm       (poly_idx) object 'Kausen\n'
    Plot       (poly_idx) object 'L1'
    Area       (poly_idx) int64 5160
    poly_area  (poly_idx) object [0       0.450979
1      25.369178
2     116...
    VV_count   (poly_idx) object [42.19559539173376]
    ...         ...
    VH_sum     (poly_idx) object [array(-683.3111709)]
    VH_mean    (poly_idx) object [array(-16.9217163)]
    VH_median  (poly_idx) object [<xarray.DataArray 'VH' ()>
array(-16.309772...
    VH_std     (poly_idx) object [<xarray.DataArray 'VH' ()>
array(2.36870381...
    VH_min     (poly_idx) object [<xarray.DataArray 'VH' ()>
array(-24.919851...
    VH_max     (poly_idx) object [<xarray.DataArray 'VH' ()>
array(-13.784737...